In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.fftpack import rfft, irfft, rfftfreq

In [2]:
microclimate = pd.read_csv("../result/микроклимат.csv", parse_dates=['Дата и время'])
soil_liter_1 = pd.read_csv("../result/liter_1.csv")
soil_liter_2 = pd.read_csv("../result/liter_2.csv")
soil_liter_4 = pd.read_csv("../result/liter_4.csv")
soil_liter_8 = pd.read_csv("../result/liter_8.csv")

Добавляем в датасет с климатом данные по свету

In [3]:
microclimate['Свет (blue, %)'] = 0
microclimate['Свет (white, %)'] = 0
microclimate['Свет (red, %)'] = 0
microclimate

,Дата и время,Температура (°C),Влажность воздуха (%),Концентрация CO2 (ppt),Время (час),"Свет (blue, %)","Свет (white, %)","Свет (red, %)"
0,2021-09-20 00:00:07,18.4,74.0,832,0.001944,0,0,0
1,2021-09-20 00:00:21,18.5,73.0,825,0.005833,0,0,0
2,2021-09-20 00:00:36,18.4,73.0,826,0.010000,0,0,0
3,2021-09-20 00:00:51,18.5,73.0,836,0.014167,0,0,0
4,2021-09-20 00:01:07,18.4,73.0,823,0.018611,0,0,0
...,...,...,...,...,...,...,...,...
358454,2021-12-03 17:42:15,22.6,49.0,894,1793.704167,0,0,0
358455,2021-12-03 17:42:44,22.6,53.0,885,1793.712222,0,0,0
358456,2021-12-03 17:43:03,22.6,52.0,865,1793.717500,0,0,0
358457,2021-12-03 17:43:29,22.6,51.0,863,1793.724722,0,0,0


In [4]:
def add_light(n, date_start, date_finish, blue=0, white=0, red=0):
    for i in range(n):
        date_start += pd.Timedelta(days=1)
        date_finish += pd.Timedelta(days=1)
        cond1 = (microclimate['Дата и время'] >= date_start)
        cond2 = (microclimate['Дата и время'] <= date_finish)
        microclimate.loc[(microclimate['Дата и время'] >= date_start) & 
                         (microclimate['Дата и время'] <= date_finish), 
                         "Свет (blue, %)"] = blue
        microclimate.loc[(microclimate['Дата и время'] >= date_start) & 
                         (microclimate['Дата и время'] <= date_finish), 
                         "Свет (white, %)"] = white
        microclimate.loc[(microclimate['Дата и время'] >= date_start) & 
                         (microclimate['Дата и время'] <= date_finish), 
                         "Свет (red, %)"] = red
    return microclimate, date_start, date_finish

In [5]:
# шаг 1
date_1 = pd.Timestamp(2021,9,20,8,0,0)
# шаг 2
date_start = date_1 + pd.Timedelta(days=10-1)
date_finish = date_start + pd.Timedelta(hours=14)
microclimate, date_start, date_finish = add_light(n=25, date_start=date_start, 
                                                  date_finish=date_finish, 
                                                  blue=30, white=70)
# шаг 3
microclimate, date_start, date_finish = add_light(n=15, date_start=date_start, 
                                                  date_finish=date_finish, 
                                                  white=70, red=50)
# шаг 4
microclimate, date_start, date_finish = add_light(n=20, date_start=date_start, 
                                                  date_finish=date_finish, 
                                                  blue=30, white=70, red=50)
# шаг 5
microclimate, date_start, date_finish = add_light(n=5, date_start=date_start, 
                                                  date_finish=date_finish, 
                                                  blue=30, white=70)

In [6]:
microclimate__ = microclimate.drop(['Время (час)'], axis=1)
_ = microclimate__.to_csv('../result/microclimate_WITH_LIGHT.csv', index=False)

#### Установка равного шага сигналов

In [7]:
time_soil = soil_liter_1["Время (час)"]
array = np.arange(1, len(time_soil)) * np.mean(np.diff(time_soil)) + time_soil[0]
time = [time_soil[0]] + array.tolist()
print(f'Шаг интерполяции => {np.mean(np.diff(time_soil))}')

Шаг интерполяции => 0.007576832968893807


In [8]:
soil_liter_1_new = np.interp(time, 
                             soil_liter_1["Время (час)"].values, 
                             soil_liter_1["1л"].values)
soil_liter_1_new_data = {"Время (час)":time, "Влажность почвы (%)":soil_liter_1_new}
soil_liter_1 = pd.DataFrame(soil_liter_1_new_data)

In [9]:
soil_liter_2_new = np.interp(time, 
                             soil_liter_2["Время (час)"].values, 
                             soil_liter_2["2л"].values)
soil_liter_2_new_data = {"Время (час)":time, "Влажность почвы (%)":soil_liter_2_new}
soil_liter_2 = pd.DataFrame(soil_liter_2_new_data)

In [10]:
soil_liter_4_new = np.interp(time, 
                             soil_liter_4["Время (час)"].values, 
                             soil_liter_4["4л"].values)
soil_liter_4_new_data = {"Время (час)":time, "Влажность почвы (%)":soil_liter_4_new}
soil_liter_4 = pd.DataFrame(soil_liter_4_new_data)

In [11]:
soil_liter_8_new = np.interp(time, 
                             soil_liter_8["Время (час)"].values, 
                             soil_liter_8["8л"].values)
soil_liter_8_new_data = {"Время (час)":time, "Влажность почвы (%)":soil_liter_8_new}
soil_liter_8 = pd.DataFrame(soil_liter_8_new_data)

In [12]:
dioxide_new = np.interp(time, 
                        microclimate["Время (час)"].values, 
                        microclimate["Концентрация CO2 (ppt)"].values)
CO2_data = {"Время (час)":time, "Концентрация CO2 (ppt)":dioxide_new}
CO2 = pd.DataFrame(CO2_data)

In [13]:
temperature_new = np.interp(time, 
                            microclimate["Время (час)"].values, 
                            microclimate["Температура (°C)"].values)
temperature_data = {"Время (час)":time, "Температура (°C)":temperature_new}
temperature = pd.DataFrame(temperature_data)

In [14]:
humidity_new = np.interp(time, 
                        microclimate["Время (час)"].values, 
                        microclimate["Влажность воздуха (%)"].values)
humidity_data = {"Время (час)":time, "Влажность воздуха (%)":humidity_new}
humidity = pd.DataFrame(humidity_data)

In [15]:
light_blue_new = np.interp(time, 
                        microclimate["Время (час)"].values, 
                        microclimate["Свет (blue, %)"].values)
light_blue_data = {"Время (час)":time, "Свет (blue, %)":light_blue_new}
light_blue = pd.DataFrame(light_blue_data)

In [16]:
light_white_new = np.interp(time, 
                        microclimate["Время (час)"].values, 
                        microclimate["Свет (white, %)"].values)
light_white_data = {"Время (час)":time, "Свет (white, %)":light_white_new}
light_white = pd.DataFrame(light_white_data)

In [17]:
light_red_new = np.interp(time, 
                        microclimate["Время (час)"].values, 
                        microclimate["Свет (red, %)"].values)
light_red_data = {"Время (час)":time, "Свет (red, %)":light_red_new}
light_red = pd.DataFrame(light_red_data)

In [18]:
soil_np = soil_liter_1.to_numpy()  
x_soil_1 = soil_np[:, 0]
y_soil_1 = soil_np[:, 1]

soil_np = soil_liter_2.to_numpy()
x_soil_2 = soil_np[:, 0]
y_soil_2 = soil_np[:, 1]

soil_np = soil_liter_4.to_numpy()
x_soil_4 = soil_np[:, 0]
y_soil_4 = soil_np[:, 1]

soil_np = soil_liter_8.to_numpy()
x_soil_8 = soil_np[:, 0]
y_soil_8 = soil_np[:, 1]

CO2_np = CO2.to_numpy()  
x_CO2 = CO2_np[:, 0]
y_CO2 = CO2_np[:, 1]

temperature_np = temperature.to_numpy()  
x_temperature = temperature_np[:, 0]
y_temperature = temperature_np[:, 1]

humidity_np = humidity.to_numpy()  
x_humidity = humidity_np[:, 0]
y_humidity = humidity_np[:, 1]

light_blue_np = light_blue.to_numpy()  
x_light_blue = light_blue_np[:, 0]
y_light_blue = light_blue_np[:, 1]

light_white_np = light_white.to_numpy()  
x_light_white = light_white_np[:, 0]
y_light_white = light_white_np[:, 1]

light_red_np = light_red.to_numpy()  
x_light_red = light_red_np[:, 0]
y_light_red = light_red_np[:, 1]

In [19]:
microclimate_data = {"Время (час)":time, 
                     "Концентрация CO2 (ppt)":y_CO2,
                     "Температура (°C)":y_temperature,
                     "Влажность воздуха (%)":y_humidity,
                     "Свет (blue, %)":y_light_blue,
                     "Свет (white, %)":y_light_white,
                     "Свет (red, %)":y_light_red}
microclimate_ = pd.DataFrame(microclimate_data)

_ = microclimate_.to_csv('../result/microclimate_NOT_cut_WITH_LIGHT.csv', index=False)

In [20]:
soil_data = {"Время (час)":time, 
                 "1л":y_soil_1,
                 "2л":y_soil_2,
                 "4л":y_soil_4,
                 "8л":y_soil_8}
soil_ = pd.DataFrame(soil_data)

# запись в файл
_ = soil_.to_csv('../result/soil_NOT_cut.csv', index=False)

#### Фурье - фильтрация (band-stop filter)

In [21]:
ranges = [108.7211, 201.0475, 322.0775, 538.4347, 662.9969, 780.8219, 
          876.145, 993.276, 1093.268, 1167.263, 1285.291, 1454.1, 1594.183]
results = [ranges[i] - ranges[i-1] for i in range(1, len(ranges)-1)]
min(results), max(results)

(73.99499999999989, 216.35720000000003)

In [22]:
range_min, range_max = 1/(max(results)-5), 1/(min(results)+5)

In [23]:
W_ = rfftfreq(len(time), d=np.mean(np.diff(time)))

# y_soil_1
spectrum_soil = rfft(y_soil_1)
cut_spectrum_soil = spectrum_soil.copy()
cut_spectrum_soil[(W_ > range_min) & (W_ < range_max)] = 0
cut_i_soil_1 = irfft(cut_spectrum_soil)

# y_soil_2
spectrum_soil = rfft(y_soil_2)
cut_spectrum_soil = spectrum_soil.copy()
cut_spectrum_soil[(W_ > range_min) & (W_ < range_max)] = 0
cut_i_soil_2 = irfft(cut_spectrum_soil)

# y_soil_4
spectrum_soil = rfft(y_soil_4)
cut_spectrum_soil = spectrum_soil.copy()
cut_spectrum_soil[(W_ > range_min) & (W_ < range_max)] = 0
cut_i_soil_4 = irfft(cut_spectrum_soil)

# y_soil_8
spectrum_soil = rfft(y_soil_8)
cut_spectrum_soil = spectrum_soil.copy()
cut_spectrum_soil[(W_ > range_min) & (W_ < range_max)] = 0
cut_i_soil_8 = irfft(cut_spectrum_soil)

# y_CO2
spectrum_CO2 = rfft(y_CO2)
cut_spectrum_CO2 = spectrum_CO2.copy()
cut_spectrum_CO2[(W_ > range_min) & (W_ < range_max)] = 0
cut_i_CO2 = irfft(cut_spectrum_CO2)

# y_temperature
spectrum_temperature = rfft(y_temperature)
cut_spectrum_temperature = spectrum_temperature.copy()
cut_spectrum_temperature[(W_ > range_min) & (W_ < range_max)] = 0
cut_i_temperature = irfft(cut_spectrum_temperature)

# y_humidity
spectrum_humidity = rfft(y_humidity)
cut_spectrum_humidity = spectrum_humidity.copy()
cut_spectrum_humidity[(W_ > range_min) & (W_ < range_max)] = 0
cut_i_humidity = irfft(cut_spectrum_humidity)

In [24]:
microclimate_cut_data = {"Время (час)":time, 
                         "Концентрация CO2 (ppt)":cut_i_CO2,
                         "Температура (°C)":cut_i_temperature,
                         "Влажность воздуха (%)":cut_i_humidity,
                         "Свет (blue, %)":y_light_blue,
                         "Свет (white, %)":y_light_white,
                         "Свет (red, %)":y_light_red}
microclimate_cut = pd.DataFrame(microclimate_cut_data)

In [25]:
# запись в файл данных микроклимата с данными по свету
_ = microclimate_cut.to_csv('../result/microclimate_cut_WITH_LIGHT_V2.csv', index=False)

In [26]:
soil_cut_data = {"Время (час)":time, 
                 "1л":cut_i_soil_1,
                 "2л":cut_i_soil_2,
                 "4л":cut_i_soil_4,
                 "8л":cut_i_soil_8}
soil_cut = pd.DataFrame(soil_cut_data)

In [27]:
# запись в файл данных по микроклимату
_ = soil_cut.to_csv('../result/soil_cut.csv', index=False)

In [28]:
microclimate_cut.describe()

,Время (час),Концентрация CO2 (ppt),Температура (°C),Влажность воздуха (%),"Свет (blue, %)","Свет (white, %)","Свет (red, %)"
count,236742.000000,236742.000000,236742.000000,236742.000000,236742.000000,236742.000000,236742.000000
mean,896.876076,839.988570,21.623106,54.993618,11.636812,35.366787,13.672849
std,517.813442,68.519576,1.778478,10.101997,14.616401,34.983407,22.277153
min,0.002569,381.397885,18.168020,30.823036,0.000000,0.000000,0.000000
25%,448.439323,801.668477,20.246011,46.881879,0.000000,0.000000,0.000000
50%,896.876076,839.881859,21.294835,54.162818,0.000000,70.000000,0.000000
75%,1345.312830,880.085509,22.695449,63.184642,30.000000,70.000000,50.000000
max,1793.749583,1236.079686,27.984084,83.681035,30.000000,70.000000,50.000000
